Retrieve sentence embeddings for on sample data. <br>
Link to the data file: https://drive.google.com/file/d/18iDySZLhAOAnZbFiD84y5laieZHqWRli/view?usp=sharing

## Install and Set ups

In [1]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 5.5 MB 26.8 MB/s 
     |████████████████████████████████| 1.3 MB 20.9 MB/s 
     |████████████████████████████████| 163 kB 41.3 MB/s 
     |████████████████████████████████| 7.6 MB 28.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ee5356f3b744ae2318b5c942cc5a7d2e00b5c626320ad2af27378efce947e143
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 199 kB 43.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=1d7f0f501db596b66344a79ee8476ed4e9ac013a89292ff069917dabf3092dab
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
!cp /content/drive/MyDrive/CSCI5253/RC_2022-06.zip ./

In [4]:
!unzip /content/RC_2022-06.zip

Archive:  /content/RC_2022-06.zip
  inflating: RC_2022-06              


### Language model

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
model = SentenceTransformer('bert-base-uncased')

Downloading:   0%|          | 0.00/437 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Retrieving

In [7]:
from pyspark.sql.functions import col

In [8]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, countDistinct

In [9]:
spark = SparkSession \
    .builder \
    .appName("sample") \
    .config("spark.driver.memory", "12g") \
    .getOrCreate()

In [10]:
reddit_df = spark.read.json("./RC_2022-06")

In [11]:
sample_df = reddit_df.limit(50000)

In [13]:
sample_df.show(5)

+-------------+--------+----------------+-------------+--------------+------------------+-----------------------------+----------------------+---------------------+------------------------+--------------------+-----------------------+-----------------+---------------+--------------------+--------------+--------------------+--------+---------+-------------------------------+----------------+---------------------+------------+----------------+-----------+-------------+--------+-------------+------+------------------+-------+------------+---------+------+--------------+----------+---------+----------+--------------------+------------+-----+------------+------------+--------+-------------+------------+-----------------------+--------------+----------------+---------------------+--------------+------------------+
|all_awardings|archived|associated_award|       author|author_cakeday|author_created_utc|author_flair_background_color|author_flair_css_class|author_flair_richtext|author_flair_tem

In [16]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import Vectors, VectorUDT

In [17]:
get_embeddings_udf = udf(lambda body : Vectors.dense(model.encode(body)), VectorUDT())

In [18]:
sample_df2 = sample_df.withColumn('embeddings', get_embeddings_udf(col('body')))

In [19]:
sample_df2.show()

+-------------+--------+----------------+---------------+--------------+------------------+-----------------------------+----------------------+---------------------+------------------------+--------------------+-----------------------+-----------------+---------------+--------------------+--------------+--------------------+--------+---------+-------------------------------+--------------------+---------------------+------------+----------------+-----------+-------------+--------+-------------+------+------------------+-------+------------+---------+------+--------------+----------+---------+----------+--------------------+------------+-----+------------+------------+--------+---------------+------------+-----------------------+--------------+----------------+---------------------+--------------+------------------+--------------------+
|all_awardings|archived|associated_award|         author|author_cakeday|author_created_utc|author_flair_background_color|author_flair_css_class|author_

In [21]:
for n in sample_df2.take(10):
  print(type(n.embeddings))
  print(n.embeddings)
  print(len(n.embeddings))

<class 'pyspark.ml.linalg.DenseVector'>
[0.10910357534885406,0.3907766342163086,0.031264178454875946,-0.2691796123981476,0.009633622132241726,-0.13542887568473816,0.5662272572517395,0.46576228737831116,-0.46532031893730164,-0.14014334976673126,0.32346341013908386,-0.2181885540485382,-0.32681113481521606,0.6442053914070129,-0.4195854961872101,0.6607042551040649,0.021576792001724243,-0.27903884649276733,0.09310872852802277,0.2751624882221222,0.009675413370132446,-0.09254587441682816,-0.36544638872146606,0.504008948802948,0.29141712188720703,0.38329723477363586,-0.258757621049881,0.12222713232040405,-0.44799986481666565,-0.02053987607359886,0.21351516246795654,0.19180478155612946,0.08155173808336258,-0.25674158334732056,-0.2335120588541031,-0.23630079627037048,-0.17543324828147888,-0.2554928660392761,-0.22166064381599426,0.36863580346107483,-0.6167173385620117,-0.3504418432712555,0.3634563982486725,0.5353845357894897,-0.2284298986196518,-0.11642984300851822,0.18994399905204773,-0.39150047

In [ ]:
del sample_df2